***LENDO OS DADOS***

In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
os.environ["SPARK_HOME"] = "/opt/spark/spark-3.5.5-bin-hadoop3"

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

25/04/13 13:04:26 WARN Utils: Your hostname, dg-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp1s0)
25/04/13 13:04:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 13:04:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/13 13:04:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

***LENDO OS DADOS***

In [11]:
path_csv = "../data_raw/2004-2021.tsv"

In [83]:
df_price_gas = spark.read.csv(path_csv,header=True,sep="\t",inferSchema=True)

In [84]:
df_price_gas.limit(5).show()

+------------+----------+------------+------------------+----------------+----------------------------+-----------------+-------------------+---------------------+--------------------+--------------------+--------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-----------------------------+
|DATA INICIAL|DATA FINAL|      REGIÃO|            ESTADO|         PRODUTO|NÚMERO DE POSTOS PESQUISADOS|UNIDADE DE MEDIDA|PREÇO MÉDIO REVENDA|DESVIO PADRÃO REVENDA|PREÇO MÍNIMO REVENDA|PREÇO MÁXIMO REVENDA|MARGEM MÉDIA REVENDA|COEF DE VARIAÇÃO REVENDA|PREÇO MÉDIO DISTRIBUIÇÃO|DESVIO PADRÃO DISTRIBUIÇÃO|PREÇO MÍNIMO DISTRIBUIÇÃO|PREÇO MÁXIMO DISTRIBUIÇÃO|COEF DE VARIAÇÃO DISTRIBUIÇÃO|
+------------+----------+------------+------------------+----------------+----------------------------+-----------------+-------------------+---------------------+--------------------+--------------------+--------------------+--

In [85]:
df_price_gas.printSchema()

root
 |-- DATA INICIAL: date (nullable = true)
 |-- DATA FINAL: date (nullable = true)
 |-- REGIÃO: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- NÚMERO DE POSTOS PESQUISADOS: integer (nullable = true)
 |-- UNIDADE DE MEDIDA: string (nullable = true)
 |-- PREÇO MÉDIO REVENDA: double (nullable = true)
 |-- DESVIO PADRÃO REVENDA: double (nullable = true)
 |-- PREÇO MÍNIMO REVENDA: double (nullable = true)
 |-- PREÇO MÁXIMO REVENDA: double (nullable = true)
 |-- MARGEM MÉDIA REVENDA: string (nullable = true)
 |-- COEF DE VARIAÇÃO REVENDA: double (nullable = true)
 |-- PREÇO MÉDIO DISTRIBUIÇÃO: string (nullable = true)
 |-- DESVIO PADRÃO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÍNIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÁXIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- COEF DE VARIAÇÃO DISTRIBUIÇÃO: string (nullable = true)



In [86]:
old_columns = df_price_gas.columns

***RENOMEANDO COLUNAS***

In [87]:
new_col_names = ["Data_inicial","Data_final","Regiao","Estado","Produto","Numero_postos_pesquisado","unidade_de_medida","preco_media_revenda","Desvio_padrao_revenda","Preco_minimo_revenda","Preco_maximo_revenda","Margem_media_revenda","Coef_variacao_revenda","Preco_medio_distribuicao","Desvio_padrao_distribuicao","Preco_minimo_distribuicao","Preco_maximo_distribuicao","Coef_variacao_distribuicao"]

In [88]:
for old_columns,new_col_names in zip(old_columns,new_col_names):
    df_price_gas = df_price_gas.withColumnRenamed(old_columns,new_col_names)

In [89]:
df_price_gas.limit(5).show()

+------------+----------+------------+------------------+----------------+------------------------+-----------------+-------------------+---------------------+--------------------+--------------------+--------------------+---------------------+------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
|Data_inicial|Data_final|      Regiao|            Estado|         Produto|Numero_postos_pesquisado|unidade_de_medida|preco_media_revenda|Desvio_padrao_revenda|Preco_minimo_revenda|Preco_maximo_revenda|Margem_media_revenda|Coef_variacao_revenda|Preco_medio_distribuicao|Desvio_padrao_distribuicao|Preco_minimo_distribuicao|Preco_maximo_distribuicao|Coef_variacao_distribuicao|
+------------+----------+------------+------------------+----------------+------------------------+-----------------+-------------------+---------------------+--------------------+--------------------+--------------------+---------------------+----

In [90]:
df_price_gas.columns

['Data_inicial',
 'Data_final',
 'Regiao',
 'Estado',
 'Produto',
 'Numero_postos_pesquisado',
 'unidade_de_medida',
 'preco_media_revenda',
 'Desvio_padrao_revenda',
 'Preco_minimo_revenda',
 'Preco_maximo_revenda',
 'Margem_media_revenda',
 'Coef_variacao_revenda',
 'Preco_medio_distribuicao',
 'Desvio_padrao_distribuicao',
 'Preco_minimo_distribuicao',
 'Preco_maximo_distribuicao',
 'Coef_variacao_distribuicao']

In [105]:
df_price_gas.createOrReplaceTempView("view_price_gas")

In [94]:
df_price_gas = spark.sql("""
        SELECT  Data_inicial,
                Data_final,
                Regiao,Produto,
                Numero_postos_pesquisado,
                round(preco_media_revenda,2)    as preco_media_revenda,
                round(Preco_minimo_revenda,2)   as Preco_minimo_revenda,
                round(Preco_maximo_revenda,2)   as Preco_maximo_revenda
        FROM    view_price_gas
    """)

In [93]:
df_price_gas.limit(5).show()

+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+
|Data_inicial|Data_final|      Regiao|         Produto|Numero_postos_pesquisado|preco_media_revenda|Preco_minimo_revenda|Preco_maximo_revenda|
+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     127|               1.29|                1.19|                1.35|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     387|               1.16|                0.89|                1.45|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     192|               1.39|                1.18|                1.76|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     162|               1.26|                1.09|                1.51|

***CRIANDO COLUNA ANO***

In [104]:
df_price_gas = spark.sql("""
        SELECT  Data_inicial,
                Data_final,
                Regiao,Produto,
                Numero_postos_pesquisado,
                round(preco_media_revenda,2)    as preco_media_revenda,
                round(Preco_minimo_revenda,2)   as Preco_minimo_revenda,
                round(Preco_maximo_revenda,2)   as Preco_maximo_revenda,
                YEAR(Data_inicial) as Ano_inicial,
                YEAR(Data_final) as Ano_final
        FROM    view_price_gas
    """)

In [106]:
spark.sql("SELECT * FROM view_price_gas").show(5)

+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|Data_inicial|Data_final|      Regiao|         Produto|Numero_postos_pesquisado|preco_media_revenda|Preco_minimo_revenda|Preco_maximo_revenda|Ano_inicial|Ano_final|
+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     127|               1.29|                1.19|                1.35|       2004|     2004|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     387|               1.16|                0.89|                1.45|       2004|     2004|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     192|               1.39|                1.18|                1.76|       2004|     2004|
|  2004-05

In [109]:
spark.sql("SELECT Max(Ano_inicial) as maximo_ano_incial ,Max(Ano_final) as maximo_ano_final FROM view_price_gas").show()

+-----------------+----------------+
|maximo_ano_incial|maximo_ano_final|
+-----------------+----------------+
|             2021|            2021|
+-----------------+----------------+



In [96]:
spark.sql("SELECT DISTINCT Regiao FROM view_price_gas").show()

+------------+
|      Regiao|
+------------+
|CENTRO OESTE|
|       NORTE|
|    NORDESTE|
|         SUL|
|     SUDESTE|
+------------+



***SEPARANDO POR REGIAO***

In [115]:
spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'CENTRO OESTE'").show(5)

+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|Data_inicial|Data_final|      Regiao|         Produto|Numero_postos_pesquisado|preco_media_revenda|Preco_minimo_revenda|Preco_maximo_revenda|Ano_inicial|Ano_final|
+------------+----------+------------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     127|               1.29|                1.19|                1.35|       2004|     2004|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     387|               1.16|                0.89|                1.45|       2004|     2004|
|  2004-05-09|2004-05-15|CENTRO OESTE|ETANOL HIDRATADO|                     192|               1.39|                1.18|                1.76|       2004|     2004|
|  2004-05

In [118]:
df_centro_oeste = spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'CENTRO OESTE'")

In [119]:
spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'NORTE'").show(5)

+------------+----------+------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|Data_inicial|Data_final|Regiao|         Produto|Numero_postos_pesquisado|preco_media_revenda|Preco_minimo_revenda|Preco_maximo_revenda|Ano_inicial|Ano_final|
+------------+----------+------+----------------+------------------------+-------------------+--------------------+--------------------+-----------+---------+
|  2004-05-09|2004-05-15| NORTE|ETANOL HIDRATADO|                      33|               1.81|                 1.5|                1.98|       2004|     2004|
|  2004-05-09|2004-05-15| NORTE|ETANOL HIDRATADO|                      15|               1.79|                1.74|                1.85|       2004|     2004|
|  2004-05-09|2004-05-15| NORTE|ETANOL HIDRATADO|                      53|               1.58|                1.28|                1.85|       2004|     2004|
|  2004-05-09|2004-05-15| NORTE|ETANOL HIDRATA

In [121]:
df_norte = spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'NORTE'")
df_nordeste = spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'NORDESTE'")
df_sul= spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'SUL'")
df_sudeste = spark.sql("SELECT * FROM view_price_gas WHERE Regiao = 'SUDESTE'")


***SALVADANDO DADOS EM PARQUET E CSV***

In [126]:
list_df = {"brasil_price_gas" : df_price_gas,
            "centro_oeste" : df_centro_oeste,
            "norte" : df_norte,
            "nordeste":df_nordeste,
            "sul": df_sul,
            "sudeste" : df_sudeste 
          }


In [136]:
for index,df in list_df.items():
    df.write.csv(f"../data_processed/csv/df_{index}",mode="overwrite",header=True)

In [137]:
for index,df in list_df.items():
    df.write.parquet(f"../data_processed/parquet/df_{index}",mode="overwrite")